Лабораторная работа 2. Обработка текста.

Задание:

Написать программу на языке программирования Python, реализующую:
1)	Предварительную обработку текста, включающую:
	-	Перевод текста в нижний регистр
	-	Удаление знаков препинания
	-	Удаление стоп-слов
	-	Удаление лишних символов
	-	Стемминг и лемматизацию
2)	Преобразование текста в мешок слов
3)	Преобразование текста в N-граммы, где N=2,4

Замечание!
Желательно провести предварительную обработку текста с использованием библиотеки NLTK.

In [104]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer # Стеммер для русского языка
from nltk.stem.porter import PorterStemmer # Стеммер для английского 
from nltk.stem import WordNetLemmatizer # Лемматайзер для английского
from nltk.corpus import wordnet
from pymystem3 import Mystem # Лемматайзер для русского языка 
import re
import string
from langdetect import detect
import logging
import os 
from datetime import datetime


In [ ]:
# Конфигурационный блок 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [106]:
log_dir = r'D:\Projects\SvetlanaDmitrievna\lab3_ii\logs'
os.makedirs(log_dir, exist_ok=True) 
log_filename = os.path.join(log_dir, f'{datetime.now().strftime("%H %M %S %Y %m %d")} nlp.log')

# Удаляем логеры, которые уже запущены по случайности 
logger = logging.getLogger()
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
    
# Настройки для логгера 
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # Вывод в консоль 
    ]
)
logger = logging.getLogger(__name__)


In [107]:
class TextPreprocessing: 
	"""
	TODO: сделать документацию. 
	"""

	def __init__(self, text, delete_punctuation_mode='string', text_normalization_method='lemma'):
		"""
		Инициализация класса TextPreprocessing.

		:param text: исходный текст для предобработки
		"""

		logger.info('Text is initializing...')

		self.text = text
		self.delete_punctuation_mode = delete_punctuation_mode
		self.language = self.__detect_language() 
		self.normalization_method = text_normalization_method

		logger.info(f'Text is initialized with parameters: delete_punctuation_mode={delete_punctuation_mode}, text_normalization_method={text_normalization_method}')
	
	# def __lang_to_simple(self) -> str:
	# 	"""
	# 	Выдаёт аббревиатуру языка.
	# 	"""
	# 	lang_mapping = {
	# 		'russian': 'ru',
	# 		'english': 'en'
	# 	}

	# 	return lang_mapping[self.language]
 
	def __detect_language(self) -> str: 
		"""
		Автоматическое определение языка с помощью библиотеки langdetect. 

		:return: Возвращается формат языка в стиле 'russian', 'english'.
		"""

		try:
			detected_lang = detect(self.text)
			
			lang_mapping = {
				'ru': 'russian',
				'en': 'english',
				# TODO: create more languages
			}

			language = lang_mapping[detected_lang]

			logger.info(f'Detected language: {language}')
			
			return language
		
		except:
			logger.critical('Error with detecting langauge.')

	def __to_lower_case(self) -> str:
		"""
		Приводит текст к нижнему регистру. 

		:return: возвращаем измененный текст. 
		"""
		
		logger.info('Text has been converted to lower case.')

		return self.text.lower()

	def __delete_punctuation_marks(self) -> str:
		"""
		Удаляет знаки препинания из предложения.

		Режимы (delete_punctuation_mode): 
		1. 're' — c помощью регулярных выражений. 
		2. 'string' — с помощью библиотеки string.  

		:return: текст без знаков препинания. 
		"""

		logger.info('Punctuation are being deleting...')

		match self.delete_punctuation_mode: 
			case 're':

				logger.info('Punctuation has been deleted.')

				return re.sub(r'[^\w\s]', '', self.text)
			
			case 'string':
				# Параметры: 
				# Первый аргумент '' символы для замены 
				# Второй аргумент '' символы на которые заменяем 
				# Третий аргумент string.punctuation символы, которые нужно удалить 
				# string.punctuation = "!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~" 
				translator = str.maketrans('', '', string.punctuation)

				logger.info('Punctuation has been deleted.')

				# translate() применяет таблицу преобразования к строке 
				return self.text.translate(translator)

	def __delete_stop_words(self) -> str: 
		"""
		Удаляет стоп-слова. 

		Стоп-слова — распространенные слова в языке, которые не несут значимой смысловой нагрузки 
		при анализе текста. 

		Зачем их удалять? 
		1. Текст становится меньше. 
		2. Фокус на важных словах. 
		"""

		logger.info('Stop words are being removed...')

		stopwords.words(self.language)

		stop_words = set(stopwords.words(self.language))
		
		words = self.text.split()

		filtered_words = [word for word in words if word not in stop_words]

		logger.info('Stop words have been removed.')
		
		return ' '.join(filtered_words)
		
	def __stemming(self) -> str: 
		"""
	Выполняет стемминг слов в тексте.
	
	Стемминг — процесс нахождения основы слова (стеммы) путем 
	отбрасывания окончаний и суффиксов. Например, для слов "рыбак", 
	"рыбачить", "рыбный" стеммой будет "рыб".
	
	:return: текст со словами, приведенными к их основам. 
	"""

		logger.info('Stemming is being performed...')
		
		try: 
			
			words = self.text.split() 
			stemmed_words = [] 

			match self.language: 
				case 'russian': 
					stemmer = SnowballStemmer("russian")
				case 'english': 
					stemmer = PorterStemmer()
				case _: 
					logger.error(f'Stemming not supported for language: {self.language}')
					return self.text

			for word in words:
				stemmed_words.append(stemmer.stem(word))
			
			logger.info('Stemming has been performed.')
			return ' '.join(stemmed_words)
			
		except Exception as e: 
			logger.error(f'Error during stemming: {str(e)}')
			return self.text
		
	def __lemming(self) -> str: 
		"""
		Выполняет лемматизацию слов в тексте.
			
		Лемматизация — процесс приведения слова к его словарной форме (лемме).
		Например, для слов "бегущий", "бегает", "бежит" леммой будет "бежать".
			
		:return: текст со словами в словарной форме
		"""

		logger.info('Lemmatizaion is being performed...')

		try: 
			
			words = self.text.split() 
			lemmatized_words = []

			match self.language: 
				case 'russian': 
						lemmatizer = Mystem()

						for word in words: 
							try: 
								# .parse(word) — библиотека pymorphy2 анализирует слово и 
								# возвращает список всех возможных морфологических разборов этого слова.
								# мы берем [0] — наиболее вероятный
								#
								# После чего делаем нормальную форму (лемму) .normal_form
								# Для существительных - именительный падеж, единственное число
								# Для глаголов - инфинитив
								# Для прилагательных - мужской род, единственное число, именительный падеж
								lemma = ''.join(lemmatizer.lemmatize(word)).strip()
								lemmatized_words.append(lemma)
							
							except Exception as e:
								logger.warning(f"Failed to lemmatize word '{word}': {e}")
								lemmatized_words.append(word)  # Сохраняем исходное слово

				case 'english':

					lemmatizer = WordNetLemmatizer()

					# Разобраться, что делает эта функция 
					def get_wordnet_pos(word):
						"""Отображает тег части речи NLTK на тег WordNet"""
						tag = nltk.pos_tag([word])[0][1][0].upper()
						tag_dict = {
								'J': wordnet.ADJ,
								'N': wordnet.NOUN,
								'V': wordnet.VERB,
								'R': wordnet.ADV
						}
						return tag_dict.get(tag, wordnet.NOUN)
					
					for word in words: 
						lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
						lemmatized_words.append(lemma)

				case _: 
					logger.error(f'Lemmatization not supported for language: {self.language}')
					return self.text
			
			logger.info('Lemmatization has been performed.')
			return ' '.join(lemmatized_words)
			
		except Exception as e: 
			logger.error(f'Error during lemmatizaion: {str(e)}')
			return self.text

	def __text_normalization(self) -> str: 
		"""
		Выбирает метод нормализации текста. 
		"""

		match self.normalization_method: 
			case 'lemma': 
				return self.__lemming()
			case 'stem': 
				return self.__stemming()
			case _: 
				logger.warning('There is no such thing as a text normalization method')
				logger.info('Returned text without normalization')
				return self.text

	def process_text(self) -> str:
		"""
		Обрабатывает текст, применяя все методы предобработки.
		
		:return: обработанный текст по всем функциям.
		""" 

		for func in [self.__to_lower_case, self.__delete_punctuation_marks, self.__delete_stop_words, self.__text_normalization]: 
				self.text = func()
		
		logger.info('Text preprocessing is done!')

		return self.text 

In [108]:
output = TextPreprocessing("Бывало ли у вас когда-нибудь такое, не знаю как у вас, но у меня точно такое было -- я читаю какую-нибудь книгу и по прошествии нескольких глав я и вспомнить не могу, о чем прочитал? Одно дело это с художественной литературой, а совсем другое с научно-популярными книгами, от которой, в какой-то степени может зависеть ваша учёная степень ну или карьера в конце концов.", delete_punctuation_mode='string', text_normalization_method='lemma').process_text()

print(output)

2025-03-22 23:43:07,459 - INFO - Text is initializing...
2025-03-22 23:43:07,464 - INFO - Detected language: russian
2025-03-22 23:43:07,466 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=lemma
2025-03-22 23:43:07,466 - INFO - Text has been converted to lower case.
2025-03-22 23:43:07,468 - INFO - Punctuation are being deleting...
2025-03-22 23:43:07,468 - INFO - Punctuation has been deleted.
2025-03-22 23:43:07,470 - INFO - Stop words are being removed...
2025-03-22 23:43:07,472 - INFO - Stop words have been removed.
2025-03-22 23:43:07,473 - INFO - Lemmatizaion is being performed...
2025-03-22 23:43:25,717 - INFO - Lemmatization has been performed.
2025-03-22 23:43:25,722 - INFO - Text preprocessing is done!


бывало когданибудь такой знать точно такой читать какуюнибудь книга прошествие несколько глава вспомнить мочь прочитывать один дело это художественный литература другой научнопопулярный книга который какойто степень зависеть ваш ученый степень карьера конец конец


In [109]:
eng_output = TextPreprocessing('Unfortunately, pymorphy2 is designed specifically for morphological analysis of Russian and does not support English. To lemmatize English text you should use other tools.').process_text()

print(eng_output)

2025-03-22 23:43:25,729 - INFO - Text is initializing...
2025-03-22 23:43:25,732 - INFO - Detected language: english
2025-03-22 23:43:25,733 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=lemma
2025-03-22 23:43:25,734 - INFO - Text has been converted to lower case.
2025-03-22 23:43:25,736 - INFO - Punctuation are being deleting...
2025-03-22 23:43:25,736 - INFO - Punctuation has been deleted.
2025-03-22 23:43:25,737 - INFO - Stop words are being removed...
2025-03-22 23:43:25,738 - INFO - Stop words have been removed.
2025-03-22 23:43:25,738 - INFO - Lemmatizaion is being performed...
2025-03-22 23:43:25,740 - INFO - Lemmatization has been performed.
2025-03-22 23:43:25,741 - INFO - Text preprocessing is done!


unfortunately pymorphy2 design specifically morphological analysis russian support english lemmatize english text use tool
